In [1]:
# Create cluster folder for the local Kudu cluster
!mkdir /tmp/cluster
# Force clear it between runs to have a clear start every time
!rm -rf /tmp/cluster/*
# Set the log level accordingly
!$KUDU_HOME/src/kudu/scripts/start_kudu.sh \
--builddir $KUDU_HOME/build/debug \
--clusterdir /tmp/cluster \
--num-masters 3 \
--master-flags "--v 2" \
--tserver-flags "--v 2"
# Wait for the servers to come up.
!sleep 3

mkdir: /tmp/cluster: File exists
/Users/mgreber/git/kudu/build/debug/bin/kudu-master
/Users/mgreber/git/kudu/build/debug/bin/kudu-tserver
Starting master-0:
  RPC  port 8764
  HTTP port 8765
  PID 2998
Starting master-1:
  RPC  port 8766
  HTTP port 8767
  PID 3000
Starting master-2:
  RPC  port 8768
  HTTP port 8769
  PID 3002
Starting tserver-0:
  RPC  port 9870
  HTTP port 9871
  PID 3009
Starting tserver-1:
  RPC  port 9872
  HTTP port 9873
  PID 3015
Starting tserver-2:
  RPC  port 9874
  HTTP port 9875
  PID 3021
ARGS
/Users/mgreber/git/kudu/build/debug/bin/kudu-master --master_addresses=127.0.0.1:8764,127.0.0.1:8766,127.0.0.1:8768 --fs_data_dirs=/tmp/cluster/master-0/data --fs_wal_dir=/tmp/cluster/master-0/wal --log_dir=/tmp/cluster/master-0/log --rpc_bind_addresses=127.0.0.1:8764 --time_source=system_unsync --unlock_unsafe_flags --webserver_interface=127.0.0.1 --webserver_port=8765 --v 2
/Users/mgreber/git/kudu/build/debug/bin/kudu-master --master_addresses=127.0.0.1:8764,127.0

In [2]:
# Create traces folder for recordings
!mkdir /tmp/traces
# Masters start recording
!curl http://localhost:8765/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6ZmFsc2V9
!curl http://localhost:8767/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6ZmFsc2V9
!curl http://localhost:8769/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6ZmFsc2V9
# Tservers start recording
!curl http://localhost:9871/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6dHJ1ZX0=
!curl http://localhost:9873/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6dHJ1ZX0=
!curl http://localhost:9875/tracing/json/begin_recording?eyJ0cmFjaW5nUmVjb3JkTW9kZSI6InJlY29yZC11bnRpbC1mdWxsIiwiY2F0ZWdvcnlGaWx0ZXIiOiIiLCJ1c2VTYW1wbGluZyI6dHJ1ZX0=                                                                
!sleep 1
# 
!python3.8 $KUDU_HOME/examples/python/basic-python-example/basic_example.py -m 127.0.0.1 127.0.0.1 127.0.0.1 -p 8764 8766 8768
!sleep 1
# End all recordings
!curl http://localhost:8765/tracing/json/end_recording_compressed --output /tmp/traces/master1.json.gz
!curl http://localhost:8767/tracing/json/end_recording_compressed --output /tmp/traces/master2.json.gz
!curl http://localhost:8769/tracing/json/end_recording_compressed --output /tmp/traces/master3.json.gz
!curl http://localhost:9871/tracing/json/end_recording_compressed --output /tmp/traces/tserver1.json.gz
!curl http://localhost:9873/tracing/json/end_recording_compressed --output /tmp/traces/tserver2.json.gz
!curl http://localhost:9875/tracing/json/end_recording_compressed --output /tmp/traces/tserver3.json.gz

import json
import gzip

def extract_trace_events(file_path):
    with gzip.open(file_path, 'rt', encoding='ISO-8859-1') as file:
        data = json.load(file)
        return data.get('traceEvents', [])

def combine_trace_events(file_paths):
    combined_events = []
    for file_path in file_paths:
        events = extract_trace_events(file_path)
        combined_events.extend(events)
    return {'traceEvents': combined_events}

def save_combined_trace_events(combined_data, output_file):
    with open(output_file, 'w') as file:
        json.dump(combined_data, file)

# Masters
master_file_paths = ['/tmp/traces/master1.json.gz', '/tmp/traces/master2.json.gz', '/tmp/traces/master3.json.gz']
# Tservers
tserver_file_paths = ['/tmp/traces/tserver1.json.gz', '/tmp/traces/tserver2.json.gz', '/tmp/traces/tserver3.json.gz']
# Masters + tservers
all_file_paths = master_file_paths + tserver_file_paths
output_file_path = '/tmp/traces/all_traces.json'
combined_data = combine_trace_events(all_file_paths)
save_combined_trace_events(combined_data, output_file_path)
print("done with trace processing")

mkdir: /tmp/traces: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18856  100 18856    0     0   925k      0 --:--:-- --:--:-- --:--:-- 1315k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18369  100 18369    0     0   965k      0 --:--:-- --:--:-- --:--:-- 1379k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18244  100 18244    0     0  1104k      0 --:--:-- --:--:-- --:--:-- 1484k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9596  100  9596    0     0   637k      0 --:--:-- --:--:-- --:--:-- 1171k
  % Total    % Recei

In [3]:
%%bash
if [[ "$OSTYPE" == "darwin"* ]]; then
    # Mac OSX
    # Applescripts to open the master and tserver logs in iterm sessions.
    osascript iterm_master_logs.applescript
    osascript iterm_tserver_logs.applescript
fi

In [4]:
!$KUDU_HOME/src/kudu/scripts/stop_kudu.sh
!sleep 2
!echo "Kudu stopped"

Kudu stopped
